In [1]:
# efetuar download do dataset previamente: https://www.kaggle.com/c/dogs-vs-cats/data

import os

WEIGHTS_DIR = './weights'
ORIGINAL_TRAIN_DIR = os.path.join('./data', 'train')
REDUCED_TRAIN_DIR = os.path.join('./data', 'train_reduced')
REDUCED_VAL_DIR = os.path.join('./data', 'val_reduced')

In [2]:
# Data preparation

import glob
import random
import shutil

os.makedirs(WEIGHTS_DIR, exist_ok=True)

N_train = 1250
N_val = 125

print (os.path.isdir(REDUCED_TRAIN_DIR))

# Remove "not" if not executed yet
if not os.path.isdir(REDUCED_TRAIN_DIR):
    for cls in ('cat', 'dog'):
        dst_train_dir = os.path.join(REDUCED_TRAIN_DIR, cls)
        dst_val_dir = os.path.join(REDUCED_VAL_DIR, cls)
        os.makedirs(dst_train_dir, exist_ok=True)
        os.makedirs(dst_val_dir, exist_ok=True)
        files = glob.glob(os.path.join(ORIGINAL_TRAIN_DIR, cls + '*'))
        random.shuffle(files)
        for f in files[:N_train]:
            shutil.copy(f, dst_train_dir)
        for f in files[N_train:N_train + N_val]:
            shutil.copy(f, dst_val_dir)

TypeError: makedirs() got an unexpected keyword argument 'exist_ok'

In [3]:
import keras
# Rede usada de 16 camadas.
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
print(keras.__version__)

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


2.1.3


In [4]:
# Importando a rede e usando o peso da imagenet, include_top=False não será incluido o topo da rede.
base_model = VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x) # (x) concatena camapadas
# Adicionando uma rede densa, uma nova camada.
x = Dense(1024, activation='relu')(x)
# Add a binary classification layer (sigmoid)
# camada de classificação
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [7]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 global_average_pooling2d_1 True
20 dense_1 True
21 dense_2 True


In [8]:
for layer in model.layers[:11]:
    layer.trainable = False

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

height, width = 224, 224
datagen = ImageDataGenerator(
    rotation_range=45.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

batch_size = 32

train_generator = datagen.flow_from_directory(
        REDUCED_TRAIN_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        REDUCED_VAL_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

Found 2500 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [ ]:
from keras.callbacks import TensorBoard

tb_callback = TensorBoard(log_dir='tb_log')

training = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        callbacks=[tb_callback])

/usr/local/lib/python3.6/site-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/5
 9/78 [==>...........................] - ETA: 3:51:09 - loss: 1.4056 - acc: 0.4757

In [ ]:
model.save('trained_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('trained_model.h5')

In [ ]:
# Inferência em uma imagem
from keras.preprocessing import image
from skimage.io import imshow
import numpy as np

img = image.load_img('data/train/dog.100.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)